In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('desafio-base1/pacientes.csv', parse_dates=['DataNasc'], encoding='iso-8859-1',quotechar='"', delimiter='|')
print(df.dtypes)

Código                       int64
Nome                        object
DataNasc            datetime64[ns]
Sexo                        object
Estado                      object
Endereco                    object
Cidade                      object
CEP                         object
Naturalidade                object
Telefone                    object
TipoTelefone                object
Profissao                   object
Pai                         object
Mae                         object
Conjuge                     object
ProfissaoConjuge            object
Cor                         object
EstadoCivil                 object
dtype: object


In [4]:
# Colocando o nome das colunas do arquivo de acordo a documentacção da iClinic
# Tem informação adicional (pode ser colocado em observation) e que falta tratamento por estar em series separadas: Conjuge, ProfissaoConjuge, Telefone, TipoTelefone
df_renamed = df.rename(
    columns = {
        "Código":"patient_id",
        "Nome": "name",
        "DataNasc":"birth_date",
        "Sexo": "gender",
        "Estado": "birth_state",
        "Endereco": "address",
        "Cidade": "city",
        "CEP": "zip_code",
        "Naturalidade": "birth_place",
        "Profissao": "occupation",
        "Pai": "patientrelatedness_father_names",
        "Mae": "patientrelatedness_mother_names",
        "Cor": "ethnicity",
        "EstadoCivil": "marital_status",
    }
)

In [5]:
df_renamed.head(8)

,patient_id,name,birth_date,gender,birth_state,address,city,zip_code,birth_place,Telefone,TipoTelefone,occupation,patientrelatedness_father_names,patientrelatedness_mother_names,Conjuge,ProfissaoConjuge,ethnicity,marital_status
0,0,Pedro Henrique Porto,NaT,M,RJ,"Quadra Caroline Dias, 482, Salgado Filho",Ramos,31924-681,Campos do Amparo,(021) 9313-9128,T,Reumatologista,Diego Silveira,Emanuella Lima,Lorenzo Silveira,Juiz ou árbitro de futebol,NaN,NaN
1,1,João Vitor da Paz,1996-01-01,M,AP,"Setor Nunes, Santa Lúcia",Nogueira,28517-970,Araújo,(021) 5202 2566,T,Fotógrafo,André Campos,Isabel Gomes,Pietra Pereira,Astrônomo,B,VI
2,2,Theo Nunes,1959-01-01,NaN,NaN,NaN,NaN,NaN,Lopes,NaN,NaN,Atendente,Eduardo Lima,Brenda Mendes,Marcos Vinicius Aragão,Armador,I,NaN
3,3,Isis da Rosa,1983-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Encarregado de manutenção predial,NaN,Rebeca da Rocha,Raquel Castro,Assistente administrativo,A,NaN
4,4,Srta. Beatriz Santos,NaT,F,MA,"Favela de Peixoto, 43, Pantanal",Barros de Moreira,59801-069,Peixoto,NaN,NaN,Psicólogo,NaN,Gabrielly da Costa,NaN,NaN,P,ES
5,5,Sofia Pires,NaT,F,TO,"Vale de Porto, 9, Mangueiras",Jesus,35356581,Oliveira da Mata,+55 51 5247 5165,T,Chapeiro,Pietro Mendes,Gabriela Mendes,Carlos Eduardo das Neves,Revisor,A,NaN
6,6,Francisco da Mota,2011-01-01,NaN,NaN,NaN,NaN,NaN,Barbosa,+55 (011) 6998 0347,T,Agrimensor,NaN,Alexia Rocha,NaN,NaN,P,VI
7,7,Renan Viana,NaT,M,SE,"Núcleo de Gonçalves, 85, Luxemburgo",Gomes,97431633,Santos,61 2746 3446,R,Agente comunitário de saúde,Diogo Moreira,Srta. Alícia Aragão,Alexandre Gomes,Gourmet,NaN,NaN


In [6]:
# Trantando a serie "gender"
# testei uma ideia originalmente, mas não foi correta
# df_renamed['gender'] = df_renamed['gender'].astype(str).str.lower()
# pelo que achei melhor utilizar list comprehension, desta forma se tiver algum outro caracter não correto ele receberá o valor NaN
# print(df_renamed['gender'].isna().sum())
df_renamed['gender'] = ['m' if x == 'M' else 'f' if x == 'F' else np.nan for x in df_renamed['gender']]
# print(df_renamed['gender'].isna().sum())
# Verifiquei que a quantidade de NaN ficou imutável após a mudança

In [7]:
# Trantando a serie "birth_state"
teste = df_renamed['birth_state'].str.contains(r'\b[A-Z]{2}')
print(teste.value_counts())
# Verifiquei se alguma informação não se adequava ao padrão estado (duas letras maiusculas), como só tive o valor True, o resto é NaN, logo está OK

True    632
Name: birth_state, dtype: int64


In [8]:
# Trantando a serie "birth_state"
# Primeiro verifiquei a quantidade de NaN para garantir que ele continuasse o mesmo
# Em seguida verifiquei qual 
# print(df_renamed['zip_code'].isna().sum())
rows_with_dashes = df_renamed['zip_code'].str.contains('-')
df_renamed['zip_code'] = [df_renamed['zip_code'][i] if x == True else df_renamed['zip_code'][i][:5]+'-'+df_renamed['zip_code'][i][5:] if x == False else np.nan for i, x in enumerate(rows_with_dashes)]
# print(df_renamed['zip_code'].isna().sum())

In [9]:
#Fazendo analise dos valores na serie TipoTelefone
print(df_renamed['TipoTelefone'].value_counts())

C    225
T    195
R    195
Name: TipoTelefone, dtype: int64


In [10]:
#Criando as colunas relativas ao tipo de phone e tratando as series 'Telefone' e 'TipoTelefone'
df_renamed['mobile_phone'] = np.nan
df_renamed['home_phone'] = np.nan
df_renamed['office_phone'] = np.nan

In [11]:
#Extraindo apenas os valores numericos dos numeros de telefone
df_renamed['Telefone'] = df_renamed['Telefone'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int).astype('str')

In [12]:
#Verificando o tipo de telefone T, trabalho
rows_t = df_renamed['TipoTelefone'].str.contains('T', na=False)
df_renamed['office_phone'] = '(' + df_renamed['Telefone'][rows_t].str[-10:-8] + ')' + df_renamed['Telefone'][rows_t].str[-8:-4] + '-' + df_renamed['Telefone'][rows_t].str[-4:]
#print(df_renamed['Telefone'][rows_t],"\n")

In [13]:
#Verificando o tipo de telefone C, celular
rows_c = df_renamed['TipoTelefone'].str.contains('C', na=False)
df_renamed['mobile_phone'] = '(' + df_renamed['Telefone'][rows_c].str[:2] + ')9' + df_renamed['Telefone'][rows_c].str[-8:-4] + '-' + df_renamed['Telefone'][rows_c].str[-4:]
#print(df_renamed['Telefone'][rows_c],"\n")

In [14]:
#Verificando o tipo de telefone R, residencial
rows_r = df_renamed['TipoTelefone'].str.contains('R', na=False)
df_renamed['home_phone'] = '(' + df_renamed['Telefone'][rows_r].str[-10:-8] + ')' + df_renamed['Telefone'][rows_r].str[-8:-4] + '-' + df_renamed['Telefone'][rows_r].str[-4:]
#print(df_renamed['Telefone'][rows_r],"\n")

In [15]:
# Tratar serie "marital_status"
print(df_renamed['marital_status'].value_counts())
#CA = casado = ma
#ES = união estável = st
#VI = Viúvo = wi
#SE = Separado = se

CA    187
ES    170
VI    145
SE    132
Name: marital_status, dtype: int64


In [16]:
rows_with_something = df_renamed['marital_status'].str.contains('CA', na=False)
df_renamed['marital_status'] = ['ma' if x == True else df_renamed['marital_status'][i] for i, x in enumerate(rows_with_something)]
rows_with_something = df_renamed['marital_status'].str.contains('ES', na=False)
df_renamed['marital_status'] = ['st' if x == True else df_renamed['marital_status'][i] for i, x in enumerate(rows_with_something)]
rows_with_something = df_renamed['marital_status'].str.contains('VI', na=False)
df_renamed['marital_status'] = ['wi' if x == True else df_renamed['marital_status'][i] for i, x in enumerate(rows_with_something)]
rows_with_something = df_renamed['marital_status'].str.contains('SE', na=False)
df_renamed['marital_status'] = ['se' if x == True else df_renamed['marital_status'][i] for i, x in enumerate(rows_with_something)]
#print(df_renamed['marital_status'])

In [17]:
# Tratar serie "ethnicity"
print(df_renamed['ethnicity'].value_counts())
#B = branca = wh
#A = amarela = ye
#P = parda = br
#N = negra = bl
#I = indegena = br?

B    132
A    131
P    127
N    127
I    114
Name: ethnicity, dtype: int64


In [18]:
rows_with_something = df_renamed['ethnicity'].str.contains('B', na=False)
df_renamed['ethnicity'] = ['wh' if x == True else df_renamed['ethnicity'][i] for i, x in enumerate(rows_with_something)]
rows_with_something = df_renamed['ethnicity'].str.contains('A', na=False)
df_renamed['ethnicity'] = ['ye' if x == True else df_renamed['ethnicity'][i] for i, x in enumerate(rows_with_something)]
rows_with_something = df_renamed['ethnicity'].str.contains('P', na=False)
df_renamed['ethnicity'] = ['br' if x == True else df_renamed['ethnicity'][i] for i, x in enumerate(rows_with_something)]
rows_with_something = df_renamed['ethnicity'].str.contains('N', na=False)
df_renamed['ethnicity'] = ['bl' if x == True else df_renamed['ethnicity'][i] for i, x in enumerate(rows_with_something)]
rows_with_something = df_renamed['ethnicity'].str.contains('I', na=False)
df_renamed['ethnicity'] = ['br' if x == True else df_renamed['ethnicity'][i] for i, x in enumerate(rows_with_something)]

In [19]:
#Tratar 'address'
#Criando as colunas relativas ao endereço do paciente 
df_renamed['number'] = np.nan
df_renamed['complement'] = np.nan
df_renamed['neighborhood'] = np.nan
df_renamed['state'] = np.nan
df_renamed['country'] = "BR"

In [38]:
# Tratamento da serie 'address'
row_with_adress = df_renamed['address'].str.split(',')
df_renamed['number'] = [np.nan if str(x) == 'nan' else x[1].strip() if len(x) == 3 else np.nan for x in row_with_adress]
df_renamed['neighborhood'] = [np.nan if str(x) == 'nan' else x[1].strip() if len(x) == 2 else x[2].strip() for x in row_with_adress]
df_renamed['address'] = [np.nan if str(x) == 'nan' else x[0] for x in row_with_adress]


In [40]:
df_renamed.head(8)

,patient_id,name,birth_date,gender,birth_state,address,city,zip_code,birth_place,Telefone,...,ethnicity,marital_status,mobile_phone,home_phone,office_phone,number,complement,neighborhood,state,country
0,0,Pedro Henrique Porto,NaT,m,RJ,Quadra Caroline Dias,Ramos,31924-681,Campos do Amparo,2193139128,...,NaN,NaN,NaN,NaN,(21)9313-9128,482,NaN,Salgado Filho,NaN,BR
1,1,João Vitor da Paz,1996-01-01,m,AP,Setor Nunes,Nogueira,28517-970,Araújo,2152022566,...,wh,wi,NaN,NaN,(21)5202-2566,NaN,NaN,Santa Lúcia,NaN,BR
2,2,Theo Nunes,1959-01-01,NaN,NaN,NaN,NaN,NaN,Lopes,NaN,...,br,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BR
3,3,Isis da Rosa,1983-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,ye,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BR
4,4,Srta. Beatriz Santos,NaT,f,MA,Favela de Peixoto,Barros de Moreira,59801-069,Peixoto,NaN,...,br,st,NaN,NaN,NaN,43,NaN,Pantanal,NaN,BR
5,5,Sofia Pires,NaT,f,TO,Vale de Porto,Jesus,35356-581,Oliveira da Mata,555152475165,...,ye,NaN,NaN,NaN,(51)5247-5165,9,NaN,Mangueiras,NaN,BR
6,6,Francisco da Mota,2011-01-01,NaN,NaN,NaN,NaN,NaN,Barbosa,5501169980347,...,br,wi,NaN,NaN,(11)6998-0347,NaN,NaN,NaN,NaN,BR
7,7,Renan Viana,NaT,m,SE,Núcleo de Gonçalves,Gomes,97431-633,Santos,6127463446,...,NaN,NaN,NaN,(61)2746-3446,NaN,85,NaN,Luxemburgo,NaN,BR


In [43]:
#Adiconando as colunas faltantes
df_renamed['gender'] = np.nan
df_renamed['cpf'] = np.nan
df_renamed['rg'] = np.nan
df_renamed['rg_issuer'] = np.nan #na documentação estava repetindo 'rg'
df_renamed['email'] = np.nan
df_renamed['email_secondary'] = np.nan
df_renamed['picture_filename'] = np.nan
df_renamed['religion'] = np.nan
df_renamed['education'] = np.nan
df_renamed['responsible'] = np.nan
df_renamed['cns'] = np.nan #na documentação estava não existe 'sms'
df_renamed['died'] = np.nan
df_renamed['death_info'] = np.nan
df_renamed['nationality'] = np.nan 
df_renamed['indication'] = np.nan
df_renamed['indication_observation'] = np.nan
df_renamed['active'] = np.nan 
df_renamed['receive_email'] = np.nan
df_renamed['observation'] = np.nan
df_renamed['healthinsurance_pack'] = np.nan 
df_renamed['tag_names'] = np.nan
df_renamed['tag_physician_id'] = np.nan

In [46]:
df_rearanged =  df_renamed.loc[:, ["patient_id","name","birth_date","gender","cpf","rg","rg_issuer","mobile_phone","home_phone","office_phone","email","email_secondary","birth_place","birth_state","zip_code","address","number","complement","neighborhood","city","state","country","picture_filename","ethnicity","marital_status","religion","occupation","education","responsible","cns","died","death_info","nationality","indication","indication_observation","active","receive_email","observation","healthinsurance_pack","patientrelatedness_mother_names","patientrelatedness_father_names","tag_names","tag_physician_id"]]

In [47]:
df_rearanged.to_csv('patient.csv',index=False, encoding='utf-8')
